In [185]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import urllib.parse

In [186]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [300]:
LOCAL = True

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [289]:
%%capture pwd
!pwd

In [290]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
uid=urllib.parse.quote(uid)
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('how-did-countries-experiencing-internal-conflict-cope-with-the-pandemic',
 300,
 500)

# Fig 1

## a

In [291]:
df = pd.read_excel("raw/fig_1_data_excel.xlsx").dropna()

In [292]:
f = "fig1a_d_rates"
f1a = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1a.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1a = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,code,continent,country,internal_conf_icrg_g_21_20 (in %),log_totaldeathspermil2021_2020,internal_conf_icrg_21minus20,general_internalconf_g
2,AGO,Africa,Angola,0.92,3.954316,0.04,0
3,ALB,Europe,Albania,-7.95,7.020870,-0.29,1
5,ARE,Asia,United Arab Emirates,-12.79,5.378006,-0.45,1
6,ARG,South America,Argentina,7.45,7.851338,0.29,0
7,ARM,Asia,Armenia,3.06,7.895760,0.13,0


In [293]:
base = alt.Chart(f1a).encode(
    x=alt.X(
        "x:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="right",
            titleAnchor="end",
            title="COVID-19 death rates per million at the end of 2021",
            titleY=-15,
            # titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
        scale=alt.Scale(
            type='log'
        )
    )
).transform_calculate(x='exp(datum.log_totaldeathspermil2021_2020)').transform_filter('datum.general_internalconf_g==1')
points = base.mark_point(fill=colors["eco-light-blue"],color=colors["eco-mid-blue"]).encode(
    y=alt.Y(
        "internal_conf_icrg_21minus20:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="Absolute change in internal conflict between 2020 and 2021",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
        # scale=alt.Scale(domain=[-1.4, 1.5]),
    )
)
labels=points.mark_text(color=colors["eco-mid-blue"],fontSize=9).encode(text='code:N')
fit = base.transform_regression(
        "log_totaldeathspermil2021_2020", "internal_conf_icrg_21minus20"
    ).transform_calculate(x2='exp(datum.log_totaldeathspermil2021_2020)')\
.mark_line(opacity=0.5).encode(x='x2:Q',y='internal_conf_icrg_21minus20:Q')
layer1 = (
    ((fit+labels).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".svg")
layer1.save("visualisation/" + f + ".png")
open("README.md", "w").write(readme)
layer1

alt.LayerChart(...)

# Fig 1

## b

In [294]:
f = "fig1b_d_change_perc"
f1b = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1b.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1b = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,code,continent,country,internal_conf_icrg_g_21_20 (in %),log_totaldeathspermil2021_2020,internal_conf_icrg_21minus20,general_internalconf_g
2,AGO,Africa,Angola,0.92,3.954316,0.04,0
3,ALB,Europe,Albania,-7.95,7.020870,-0.29,1
5,ARE,Asia,United Arab Emirates,-12.79,5.378006,-0.45,1
6,ARG,South America,Argentina,7.45,7.851338,0.29,0
7,ARM,Asia,Armenia,3.06,7.895760,0.13,0


In [295]:
base = alt.Chart(f1b).encode(
    x=alt.X(
        "x:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="right",
            titleAnchor="end",
            title="COVID-19 death rates per million at the end of 2021",
            titleY=-15,
            # titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
        scale=alt.Scale(
            type='log'
        )
    )
).transform_calculate(x='exp(datum.log_totaldeathspermil2021_2020)').transform_filter('datum.general_internalconf_g==1')
points = base.mark_point(fill=colors["eco-light-blue"],color=colors["eco-mid-blue"]).encode(
    y=alt.Y(
        "internal_conf_icrg_g_21_20 (in %):Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="Relative change in internal conflict between 2020 and 2021 (%)",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
        scale=alt.Scale(domain=[-25, 30]),
    )
)
labels=points.mark_text(color=colors["eco-mid-blue"],fontSize=9).encode(text='code:N')
fit = base.transform_regression(
        "log_totaldeathspermil2021_2020", "internal_conf_icrg_g_21_20 (in %)"
    ).transform_calculate(x2='exp(datum.log_totaldeathspermil2021_2020)')\
.mark_line(opacity=0.5).encode(x='x2:Q',y='internal_conf_icrg_g_21_20 (in %):Q')
layer1 = (
    ((fit+labels).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".svg")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

# Fig 2

In [337]:
df1 = pd.read_csv("raw/wpd_datasets.csv",skiprows=1)
df2 = pd.read_csv("raw/gdp.csv",header=None)
df1.columns=['x1','y1','x2','y2','x3','y3']
df2.columns=['gx','gy']
df=df1.join(df2)

df0=pd.DataFrame(list(range(0,66))+list(df['x2'].values)).sort_values(by=0).dropna().set_index(0)
df2=df0.join(f2.set_index('x2')[['y2']].dropna()).interpolate().loc[list(range(0,66))]

df0=pd.DataFrame(list(range(0,66))+list(df['x3'].values)).sort_values(by=0).dropna().set_index(0)
df2['y3']=df0.join(f2.set_index('x3')[['y3']].dropna()).interpolate().loc[list(range(0,66))]
df2.index.name='x'

df=df.join(df2.dropna().reset_index(),rsuffix='r')

In [338]:
f = "fig2_effects"
f2 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,x1,y1,x2,y2,x3,y3,gx,gy,x,y2r,y3r
0,0.112776,3.219650,0.741422,4.408506,0.115946,1.964149,2.980519,51.747312,1.0,4.339084,1.947076
1,0.741422,3.176261,1.998713,4.269661,2.202978,1.912932,9.189933,18.413978,2.0,4.247967,1.930004
2,1.370067,3.111178,2.391617,4.226272,7.246637,1.631242,15.647723,14.045699,3.0,4.152511,1.865984
3,1.998713,3.067789,3.570327,4.078750,11.304753,1.375160,22.478079,5.309140,4.0,3.996311,1.819035
4,2.627359,3.028739,3.963231,4.035361,15.420843,1.003841,29.060058,7.056452,5.0,3.957261,1.772087


In [347]:
base = alt.Chart(f2).encode(
    x=alt.X(
        "gx:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="right",
            titleAnchor="end",
            title="Macrofinanacial packages as % of GDP",
            titleY=-30,
            # titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        )
    )
)
bars = base.mark_bar(color=colors["eco-light-blue"],width=25).encode(
    y=alt.Y(
        "gy:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="Distribution over countries (%)",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
        # scale=alt.Scale(domain=[-25, 30]),
    )
)
line=base.mark_line(color=colors['eco-mid-blue']).encode(x=alt.X(
        "x1:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="right",
            titleAnchor="end",
            title="",
            titleY=-30,
            # titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        )
    ), y=alt.Y(
        "y1:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title=["COVID-19 death rates per million on", "growth rates civil disorder between 2020 and 2021"," with 90% confidence band"],
            titleX=-5,
            titleY=-35,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
        # scale=alt.Scale(domain=[-25, 30]),
    ))
area=base.mark_area(color=colors['eco-mid-blue'],opacity=0.1).encode(x=alt.X(
        "x:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="right",
            titleAnchor="end",
            title="",
            titleY=-30,
            # titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        )
    ), y=alt.Y(
        "y2r:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title=["COVID-19 death rates per million on", "growth rates civil disorder between 2020 and 2021"," with 90% confidence band"],
            titleX=-5,
            titleY=-35,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
        # scale=alt.Scale(domain=[-25, 30]),
    ),y2='y3r:Q')
layer1 = (
    ((area+line).properties(height=300, width=400))
)
layer1=alt.vconcat(bars.properties(height=300, width=400),layer1.properties(height=200, width=400)).resolve_axis(
    x='shared'
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".svg")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

WARN Axes cannot be shared in concatenated or repeated views yet (https://github.com/vega/vega-lite/issues/2415).
WARN Axes cannot be shared in concatenated or repeated views yet (https://github.com/vega/vega-lite/issues/2415).


alt.VConcatChart(...)

In [349]:
layer1=bars
layer1.save("visualisation/" + f + "_bars_only.json")
layer1.save("visualisation/" + f + "_bars_only.svg")
layer1.save("visualisation/" + f + "_bars_only.png")
layer1

alt.Chart(...)